# 📊 Library Status Summary for Day 04 Notebook

## ✅ Working Libraries
The following libraries have been successfully verified:
- **NumPy**: Version 1.24.0 (downgraded for compatibility)  
- **Pandas**: Version 2.2.3
- **Matplotlib**: Version 3.10.3
- **RDKit**: Version 2025.03.2
- **ASE**: Version 3.25.0
- **PySCF**: Version 2.9.0
- **DeepChem**: Version 2.8.0 (verified working with current NumPy)
- **PyTorch**: Version 2.2.2
- **scikit-learn**: Version 1.7.0

## ❌ Missing Libraries
- **Psi4**: Not installed (quantum chemistry package)

## 📝 Important Notes
1. **NumPy Downgrade**: NumPy has been downgraded to 1.24.0 for compatibility with DeepChem.
2. **Psi4 Fallback**: The notebook has mock implementations when Psi4 is not available.

## 🧪 Psi4 Installation Options

### Option 1: Using Miniforge (Recommended)
Miniforge is a minimal conda distribution that works better than standard Anaconda for scientific packages:

```bash
# Download and install Miniforge
curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-MacOSX-x86_64.sh
bash Miniforge3-MacOSX-x86_64.sh -b -p $HOME/miniforge3

# Add to PATH (for zsh)
echo 'export PATH="$HOME/miniforge3/bin:$PATH"' >> ~/.zshrc
source ~/.zshrc

# Create environment and install Psi4
mamba create -n psi4env python=3.8 -y
mamba activate psi4env
mamba install -c conda-forge psi4=1.9.1 numpy pandas matplotlib rdkit ase pyscf -y
```

### Option 2: Using Docker
Docker provides a ready-to-use container with Psi4 pre-installed:

```bash
# Install Docker Desktop first from: https://www.docker.com/products/docker-desktop/

# Pull Psi4 image
docker pull psi4/psi4:latest

# Run with current directory mounted as /work
docker run -it -v "$(pwd)":/work -w /work psi4/psi4:latest

# For Jupyter Notebook access
docker run -it -v "$(pwd)":/work -w /work -p 8888:8888 psi4/psi4:latest jupyter notebook --ip=0.0.0.0 --no-browser
```

### Option 3: Using Standard Conda
If you prefer standard Conda and are having issues with missing libraries:

```bash
# Fix libarchive issue
brew install libarchive
sudo ln -s $(brew --prefix libarchive)/lib/libarchive.19.dylib /usr/local/lib/

# Create environment and install Psi4
conda create -n psi4env python=3.8 -y
conda activate psi4env
conda install -c conda-forge psi4 -y
```

### Option 4: Continue Without Psi4
The notebook will run with mock calculations for demonstration purposes. Look for the fallback implementations throughout the notebook that activate when Psi4 is not detected.

In [ ]:
# Check for Psi4 installation and set up fallback mechanisms if needed

import sys
import subprocess
import warnings
from importlib.util import find_spec

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

# ANSI color codes for prettier output
GREEN = "\033[92m"
RED = "\033[91m"
YELLOW = "\033[93m"
BLUE = "\033[94m"
BOLD = "\033[1m"
END = "\033[0m"

# Check if Psi4 is available
print(f"{BOLD}🔍 Checking for Psi4 installation...{END}")
psi4_available = find_spec("psi4") is not None

# Set up global variable that can be used by other cells
global PSI4_AVAILABLE
PSI4_AVAILABLE = psi4_available

if psi4_available:
    print(f"{GREEN}✅ Psi4 is installed in this environment!{END}")
    print("\nRunning a simple verification test...")
    
    try:
        import psi4
        print(f"Psi4 version: {psi4.__version__}")
        
        # Set memory limit for calculation
        psi4.set_memory('500 MB')
        
        # Define hydrogen molecule geometry
        h2 = psi4.geometry("""
        0 1
        H
        H 1 0.9
        """)
        
        # Set up calculation output
        psi4.set_output_file('h2_test_output.txt')
        
        # Perform energy calculation
        energy = psi4.energy('scf/cc-pvdz')
        
        print(f"{GREEN}✅ Calculation successful!{END}")
        print(f"H2 SCF energy: {energy} Hartree")
        print("\nPsi4 is correctly installed and working!")
    
    except Exception as e:
        print(f"{RED}❌ Error while running Psi4: {str(e)}{END}")
        print("There might be issues with your Psi4 installation.")

else:
    print(f"{RED}❌ Psi4 is not installed in this environment{END}")
    print(f"{YELLOW}\nSetting up fallback mechanisms...{END}")
    
    # Check for PySCF as the primary fallback
    pyscf_available = find_spec("pyscf") is not None
    if pyscf_available:
        print(f"{GREEN}✅ PySCF is available and will be used as the primary fallback{END}")
        try:
            import pyscf
            print(f"PySCF version: {pyscf.__version__}")
            
            # Simple test to verify PySCF is working
            from pyscf import gto, scf
            mol = gto.M(atom='H 0 0 0; H 0 0 0.74', basis='cc-pvdz')
            mf = scf.RHF(mol)
            print(f"{GREEN}✅ PySCF is verified working{END}")
        except Exception as e:
            print(f"{RED}❌ Error with PySCF: {str(e)}{END}")
    else:
        print(f"{RED}❌ PySCF is not available as a fallback{END}")
    
    # Check for other critical dependencies
    print(f"\n{BOLD}Checking for critical fallback dependencies:{END}")
    for lib in ["numpy", "scipy", "matplotlib"]:
        if find_spec(lib):
            print(f"{GREEN}✅ {lib} is available{END}")
        else:
            print(f"{RED}❌ {lib} is missing - needed for fallbacks{END}")
    
    print(f"\n{YELLOW}The notebook will continue using fallback implementations.{END}")
    print("To install Psi4, refer to the installation instructions at the beginning of this notebook.")

print(f"\n{BLUE}{BOLD}Setting up standardized quantum chemistry interface...{END}")
print(f"{BLUE}This will automatically use the best available methods{END}")

# Create a unified quantum chemistry interface
def setup_quantum_interface():
    """Set up a unified interface for quantum chemistry calculations"""
    
    class QuantumInterface:
        def __init__(self):
            self.psi4_available = psi4_available
            self.pyscf_available = find_spec("pyscf") is not None
            self.backend = "psi4" if psi4_available else "pyscf" if self.pyscf_available else "mock"
            print(f"Quantum interface initialized using {self.backend} backend")
            
        def calculate_energy(self, mol_string, method="HF", basis="cc-pvdz"):
            """Calculate single point energy using best available method"""
            print(f"Calculating energy with {method}/{basis} using {self.backend} backend")
            
            if self.backend == "psi4":
                # Use real Psi4
                import psi4
                mol = psi4.geometry(mol_string)
                psi4.set_options({"basis": basis, "reference": "RHF"})
                energy = psi4.energy(f"{method}/{basis}")
                return energy
                
            elif self.backend == "pyscf":
                # Use PySCF
                from pyscf import gto, scf
                mol = gto.M(atom=mol_string, basis=basis)
                mf = scf.RHF(mol)
                energy = mf.kernel()
                return energy
                
            else:
                # Use mock implementation
                print(f"{YELLOW}⚠️ Using mock implementation for demonstration{END}")
                # Return representative values for common molecules
                if "h2" in mol_string.lower() and "o" not in mol_string.lower():
                    return -1.1371  # H2 at HF/cc-pVDZ 
                elif "h2o" in mol_string.lower():
                    return -76.0266  # H2O at HF/cc-pVDZ
                else:
                    return -10.0000  # Default mock value
    
    # Return the interface
    return QuantumInterface()

# Create global quantum interface that can be used by other cells
global quantum
quantum = setup_quantum_interface()

# 📦 Library Installation and Verification

## Current Status

| Library | Status | Notes |
|---------|--------|-------|
| **PySCF** | ✅ Installed | Version 2.9.0 |
| **ASE** | ✅ Installed | Version 3.25.0 |
| **RDKit** | ✅ Installed | Version 2025.3.2 |
| **DeepChem** | ✅ Installed | Version 2.8.0 (working with NumPy 1.24.0) |
| **Psi4** | ❌ Not installed | Not available via pip; requires conda or Docker |

## Detailed Analysis

### Psi4 Installation Issues

A thorough investigation revealed:

1. **Pip Installation**: Psi4 is not distributed via pip for macOS with Python 3.11
2. **Conda Issues**: The current conda installation has a missing library (`libarchive.19.dylib`)
3. **Solution Options**: Multiple paths exist: Miniforge, Docker, or fixing conda

### Fixed: DeepChem & NumPy Compatibility 

DeepChem has been verified working with NumPy 1.24.0:

```python
# This has been applied successfully
# pip install numpy==1.24.0
# pip install deepchem
```

### Psi4 Verification (After Installation)

Once Psi4 is installed, you can verify it works with this test:

```python
import psi4
print(f"Psi4 version: {psi4.__version__}")

# Simple test calculation
psi4.set_memory('500 MB')
h2 = psi4.geometry("""
0 1
H
H 1 0.9
""")
psi4.set_output_file('h2_test.txt')
energy = psi4.energy('scf/cc-pvdz')
print(f"H2 energy: {energy}")
```

For a full installation guide, refer to the detailed instructions at the start of this notebook.
```

## Important Notes

1. **Fallback Mechanisms**: The notebook has fallback mechanisms for when Psi4 is not available, which will use mock calculations for demonstration purposes.

2. **Docker Alternative**: You can use the provided Docker setup which includes all required dependencies:
   ```bash
   docker build -t chemml .
   docker run -p 8888:8888 -v $(pwd):/app chemml
   ```

3. **Compatibility**: Some parts of the notebook may run with reduced functionality if certain packages are unavailable or have compatibility issues.

In [ ]:
# Comprehensive Library Check for Day 04 Quantum Chemistry Notebook
# This cell will check all required and optional libraries and display their status

import sys
import importlib
from importlib.util import find_spec
import platform
import pandas as pd
import warnings
from IPython.display import display, Markdown, HTML

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

# ANSI color codes for terminal and HTML colors for display
class Colors:
    # HTML colors
    GREEN = "#00AA00"
    RED = "#AA0000"
    YELLOW = "#AA5500"
    BLUE = "#0000AA"
    
    # Terminal ANSI codes
    BOLD = '\x1b[1m'
    END = '\x1b[0m'
    BLUE_TERM = '\x1b[94m'
    GREEN_TERM = '\x1b[92m'
    RED_TERM = '\x1b[91m'
    YELLOW_TERM = '\x1b[93m'

# Define HTML colors for display
HTML_GREEN = Colors.GREEN
HTML_RED = Colors.RED
HTML_YELLOW = Colors.YELLOW

def check_library(name, required=True, fallback=None):
    """Check if a library is installed and return its status information."""
    try:
        # Special handling for RDKit
        if name == "rdkit":
            if find_spec(name):
                import rdkit
                from rdkit import rdBase
                version = rdBase.rdkitVersion
            else:
                raise ImportError("Not found")
        else:
            if find_spec(name):
                # Try to import and get version
                module = importlib.import_module(name)
                if hasattr(module, "__version__"):
                    version = module.__version__
                elif hasattr(module, "version"):
                    version = module.version
                else:
                    version = "Unknown version"
            else:
                raise ImportError("Not found")
                
        console_prefix = Colors.GREEN_TERM if required else Colors.BLUE_TERM
        return {
            "name": name, 
            "installed": True,
            "version": version,
            "status": "✅ Installed",
            "console_status": f"{console_prefix}✅ Installed{Colors.END}",
            "required": required,
            "fallback": fallback
        }
        
    except ImportError:
        console_prefix = Colors.RED_TERM if required else Colors.YELLOW_TERM
        status = "❌ Missing"
        console_status = f"{console_prefix}❌ Not Installed{Colors.END}"
        return {
            "name": name, 
            "installed": False,
            "version": "Not installed",
            "status": status,
            "console_status": console_status, 
            "required": required,
            "fallback": fallback
        }
    except Exception as e:
        console_prefix = Colors.YELLOW_TERM
        return {
            "name": name, 
            "installed": False,
            "version": f"Error: {str(e)}",
            "status": "⚠️ Error",
            "console_status": f"{console_prefix}⚠️ Error: {str(e)}{Colors.END}",
            "required": required,
            "fallback": fallback
        }

# Print header
print(f"{Colors.BLUE_TERM}{Colors.BOLD}{'=' * 50}{Colors.END}")
print(f"{Colors.BLUE_TERM}{Colors.BOLD} COMPREHENSIVE LIBRARY CHECK FOR QUANTUM CHEMISTRY {Colors.END}")
print(f"{Colors.BLUE_TERM}{Colors.BOLD}{'=' * 50}{Colors.END}")
print(f"Python version: {platform.python_version()}")
print(f"Platform: {platform.platform()}")
print(f"Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Check key libraries
key_libraries = ["numpy", "pandas", "matplotlib", "scipy", "rdkit", "ase", "pyscf", "psi4", "deepchem", "torch", "sklearn"]
library_status = {lib: check_library(lib) for lib in key_libraries}

# Display library information
display(Markdown("## 📚 Library Status"))

# Working and missing libraries
working_libs = [lib for lib, info in library_status.items() if info["installed"]]
missing_libs = [lib for lib, info in library_status.items() if not info["installed"]]

if working_libs:
    display(Markdown("### ✅ Working Libraries"))
    display(Markdown(", ".join([f"**{lib}**" for lib in working_libs])))

if missing_libs:
    display(Markdown("### ❌ Missing Libraries"))
    display(Markdown(", ".join([f"**{lib}**" for lib in missing_libs])))

# Check NumPy version for DeepChem compatibility
numpy_version = library_status["numpy"]["version"]
if numpy_version.startswith("2."):
    display(Markdown("### ⚠️ NumPy Compatibility Warning"))
    display(Markdown("NumPy 2.x may have compatibility issues with DeepChem. Consider downgrading to NumPy 1.24.0 for full compatibility."))

# Check Psi4 status
if library_status["psi4"]["status"] == "❌ Missing":
    display(Markdown("## 🧪 Quantum Chemistry Setup"))
    if library_status["pyscf"]["status"] != "❌ Missing":
        display(Markdown("✅ **PySCF is being used** as an alternative to Psi4 for quantum chemistry calculations."))
    else:
        display(Markdown("⚠️ **Neither Psi4 nor PySCF is installed.** Only mock calculations are available."))
        
    display(Markdown("""
### 🔧 Psi4 Installation Recommendation

For full quantum chemistry functionality, consider installing Psi4 using one of the methods described at the beginning of this notebook:

1. **Using Miniforge** (most reliable)
2. **Using Docker** (pre-configured environment)
3. **Using standard Conda** (may require additional setup)
    """))

# Provide recommendation for future work
display(Markdown("## 🚀 Recommendations for Future Work"))
display(Markdown("""
1. **Install missing libraries** especially Psi4 for full quantum chemistry capabilities
2. **Explore more advanced methods** like coupled cluster (CC) and density functional theory (DFT)
3. **Try more ML models** beyond those demonstrated in this notebook
4. **Combine quantum chemistry with deep learning** for property prediction
    """))

# Summary statement
display(Markdown("## 🎯 Final Notebook Status"))

# Calculate functionality metrics
total_libs = len(library_status)
working_count = sum(1 for info in library_status.values() if info["installed"])
functionality_pct = (working_count / total_libs) * 100

# Show appropriate message based on status
if functionality_pct >= 90:
    display(Markdown(f"✅ **Excellent!** {functionality_pct:.1f}% of libraries are installed. The notebook should work with full or near-full functionality."))
elif functionality_pct >= 75:
    display(Markdown(f"✅ **Good!** {functionality_pct:.1f}% of libraries are installed. Most features work, with some using fallback mechanisms."))
else:
    display(Markdown(f"⚠️ **Limited functionality:** Only {functionality_pct:.1f}% of libraries are installed. Consider using Docker for a complete environment."))

# Show Docker recommendation for comprehensive setup
display(Markdown("""
### 🐳 Docker Recommendation

For the most complete and hassle-free setup, use the provided Docker container:

```bash
docker build -t chemml .
docker run -p 8888:8888 -v $(pwd):/app chemml
```

This container includes all required libraries properly configured, including Psi4.
"""))

# 🧪 Quantum Chemistry Experiments

Now that we've verified our library setup, let's proceed with the quantum chemistry experiments. This notebook will automatically adapt to use the best available methods based on your environment.

In [ ]:
# Basic Quantum Chemistry Calculations

import numpy as np
import matplotlib.pyplot as plt
from importlib.util import find_spec
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

print("🧪 Basic Quantum Chemistry - Single Point Calculation")
print("=" * 60)

# Define our test molecule - hydrogen molecule
h2_geometry = """
H 0.0 0.0 0.0
H 0.0 0.0 0.74  # Bond length in Angstroms
"""

# Function to run calculation with best available method
def calculate_energy(geometry, basis="cc-pvdz"):
    """Calculate energy using the best available quantum chemistry package"""
    
    # Check for Psi4 first
    if find_spec("psi4"):
        print("Using Psi4 for calculation...")
        import psi4
        
        # Create molecule
        mol = psi4.geometry(geometry)
        
        # Set options
        psi4.set_options({"basis": basis})
        
        # Run calculation
        energy = psi4.energy("scf")
        return energy, "psi4"
        
    # Fall back to PySCF
    elif find_spec("pyscf"):
        print("Using PySCF for calculation...")
        from pyscf import gto, scf
        
        # Create molecule
        mol = gto.M(atom=geometry, basis=basis, verbose=0)
        
        # Run calculation
        mf = scf.RHF(mol)
        energy = mf.kernel()
        return energy, "pyscf"
        
    # Last resort - mock calculation
    else:
        print("Using mock calculation (no quantum chemistry packages available)")
        # Return representative value for H2 at this level of theory
        return -1.1371, "mock"

# Run calculation
energy, method = calculate_energy(h2_geometry)

print(f"\nResults:")
print(f"  Method: {method}")
print(f"  Basis: cc-pvdz")
print(f"  Energy: {energy:.6f} Hartree")
print(f"  Energy: {energy*627.5:.2f} kcal/mol")  # Convert to kcal/mol

# Add a visualization
print("\nVisualizing H2 molecule...")
try:
    # First attempt with RDKit and py3Dmol if available
    from rdkit import Chem
    from rdkit.Chem import AllChem
    import py3Dmol
    
    # Create RDKit molecule
    h2 = Chem.MolFromSmiles("[H][H]")
    AllChem.EmbedMolecule(h2)
    
    # Visualize in 3D
    view = py3Dmol.view(width=400, height=300)
    view.addModel(Chem.MolToMolBlock(h2), "mol")
    view.setStyle({'stick': {}})
    view.zoomTo()
    view.show()
    print("3D visualization generated above.")
except ImportError:
    # Fall back to matplotlib visualization
    print("RDKit or py3Dmol not available, using simple matplotlib visualization")
    plt.figure(figsize=(4, 2))
    plt.plot([0, 0], [0, 0], 'o', markersize=12, color='grey')
    plt.plot([0.74, 0.74], [0, 0], 'o', markersize=12, color='grey')
    plt.plot([0, 0.74], [0, 0], '-', linewidth=5, color='grey', alpha=0.5)
    plt.text(-0.05, -0.1, "H", fontsize=14)
    plt.text(0.74 - 0.05, -0.1, "H", fontsize=14)
    plt.title("H₂ Molecule (Bond length = 0.74 Å)")
    plt.xlim(-0.2, 0.9)
    plt.ylim(-0.2, 0.2)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
# Try to showcase the electron density if PySCF is available
if find_spec("pyscf"):
    try:
        print("\nCalculating electron density with PySCF...")
        from pyscf import gto, scf, tools
        from pyscf.tools import cubegen
        import numpy as np
        
        # Create molecule
        mol = gto.M(atom=h2_geometry, basis='cc-pvdz', verbose=0)
        mf = scf.RHF(mol)
        mf.kernel()
        
        # Calculate electron density on a grid
        nx, ny, nz = 40, 40, 40  # Grid points
        grid = np.mgrid[-2:2:nx*1j, -2:2:ny*1j, -2:2:nz*1j]
        coords = np.vstack([grid[0].ravel(), grid[1].ravel(), grid[2].ravel()]).T
        
        # Get density on grid
        density = tools.cubegen.density(mol, coords, mf.make_rdm1())
        density = density.reshape(nx, ny, nz)
        
        # Plot a 2D slice of the density
        plt.figure(figsize=(6, 5))
        plt.contourf(grid[0][:,:,nz//2], grid[1][:,:,nz//2], density[:,:,nz//2], 
                    levels=50, cmap='viridis')
        plt.colorbar(label='Electron Density')
        plt.plot([0, 0.74], [0, 0], 'o-', color='white')
        plt.title('H₂ Electron Density (2D slice)')
        plt.xlabel('Å')
        plt.ylabel('Å')
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print(f"Could not generate electron density plot: {str(e)}")
        
print("\nBasic quantum chemistry calculation complete!")

In [ ]:
# Potential Energy Surface Scan for H2

import numpy as np
import matplotlib.pyplot as plt
from importlib.util import find_spec
import warnings
from scipy.optimize import curve_fit

# Suppress warnings
warnings.filterwarnings('ignore')

print("🧪 Potential Energy Surface Scan for H₂")
print("=" * 60)

# Define range of bond distances to scan (in Angstroms)
bond_distances = np.linspace(0.4, 2.0, 16)  # 16 points from 0.4 to 2.0 Å
energies = []
method_used = None

# Progress bar setup
from IPython.display import display, clear_output
import time

print(f"Scanning {len(bond_distances)} bond distances from {bond_distances[0]:.2f} Å to {bond_distances[-1]:.2f} Å")
print("This may take a few minutes...")

# Get energies for each bond distance
for i, dist in enumerate(bond_distances):
    # Show progress
    progress = (i+1) / len(bond_distances) * 100
    print(f"Progress: {progress:.1f}% - Computing distance {dist:.2f} Å", end='\r')
    
    # Create geometry string with current bond distance
    geometry = f"""
    H 0.0 0.0 0.0
    H 0.0 0.0 {dist}
    """
    
    # Calculate energy
    energy, method = calculate_energy(geometry)
    energies.append(energy)
    
    if method_used is None:
        method_used = method

print("\nPotential Energy Surface calculated using", method_used)

# Convert to numpy array for easier manipulation
energies = np.array(energies)

# Find equilibrium bond length
eq_index = np.argmin(energies)
eq_distance = bond_distances[eq_index]
eq_energy = energies[eq_index]

print(f"Equilibrium bond length: {eq_distance:.4f} Angstroms")
print(f"Minimum energy: {eq_energy:.6f} Hartree ({eq_energy*627.5:.2f} kcal/mol)")

# Try to fit the data to a Morse potential
try:
    # Define Morse potential function: D_e * (1 - exp(-a*(r - r_e)))^2 + E_0
    def morse_potential(r, D_e, a, r_e, E_0):
        return D_e * (1 - np.exp(-a * (r - r_e)))**2 + E_0

    # Initial parameter guesses
    p0 = [0.1, 1.0, eq_distance, min(energies)]
    
    # Fit the data to the Morse potential
    params, _ = curve_fit(morse_potential, bond_distances, energies, p0=p0)
    
    # Extract parameters
    D_e, a, r_e, E_0 = params
    
    # Calculate dissociation energy in kcal/mol
    D_e_kcal = D_e * 627.5
    
    print("\nMorse Potential Parameters:")
    print(f"Dissociation energy (D_e): {D_e:.4f} Hartree ({D_e_kcal:.2f} kcal/mol)")
    print(f"Equilibrium bond length (r_e): {r_e:.4f} Angstroms")
    print(f"Curvature parameter (a): {a:.4f}")
    
    # Generate smooth curve for plotting
    r_range = np.linspace(min(bond_distances), max(bond_distances), 100)
    morse_energies = morse_potential(r_range, D_e, a, r_e, E_0)
    
    # Plot both raw data and fitted curve
    plt.figure(figsize=(10, 6))
    plt.plot(bond_distances, energies, 'o', markersize=8, label='Calculated Points')
    plt.plot(r_range, morse_energies, '-', linewidth=2, label='Morse Potential Fit')
    plt.axvline(x=r_e, color='red', linestyle='--', alpha=0.7, 
                label=f'Equilibrium (r_e = {r_e:.4f} Å)')
    
    # Convert to relative energies in kcal/mol for second y-axis
    relative_energies_kcal = (energies - min(energies)) * 627.5
    
    # Add second y-axis for kcal/mol
    ax1 = plt.gca()
    ax2 = ax1.twinx()
    ax2.set_ylabel('Relative Energy (kcal/mol)', color='green')
    ax2.plot(bond_distances, relative_energies_kcal, 'g^', alpha=0)  # Invisible plot to set the scale
    ax2.tick_params(axis='y', labelcolor='green')
    
    plt.xlabel('H-H Bond Distance (Angstroms)', fontsize=12)
    plt.ylabel('Energy (Hartree)', fontsize=12)
    plt.title('H₂ Potential Energy Surface with Morse Potential Fit', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.legend(loc='upper right')
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    # If curve fitting fails, just plot the raw data
    print(f"\nCould not fit to Morse potential: {str(e)}")
    
    # Convert to relative energies in kcal/mol
    relative_energies = (energies - np.min(energies)) * 627.5  # Convert to kcal/mol
    
    # Plot potential energy surface
    plt.figure(figsize=(10, 6))
    plt.plot(bond_distances, relative_energies, 'o-', linewidth=2, markersize=8)
    plt.axvline(x=eq_distance, color='red', linestyle='--', alpha=0.7, 
                label=f'Equilibrium bond length: {eq_distance:.4f} Å')
    plt.xlabel('H-H Bond Distance (Angstroms)', fontsize=12)
    plt.ylabel('Relative Energy (kcal/mol)', fontsize=12)
    plt.title('H₂ Potential Energy Surface', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()

# Save data for later use
np.savez('h2_pes_data.npz', 
         distances=bond_distances,
         energies=energies,
         method=method_used)

print("\nData saved to h2_pes_data.npz for later use")

# Analysis of vibrational frequency
try:
    # For a diatomic molecule near equilibrium, we can approximate with a harmonic oscillator
    # E = 0.5 * k * (r - r_e)^2
    # Let's use points near equilibrium to estimate k
    
    # Get points near equilibrium
    window = 3  # Number of points on each side of equilibrium
    start_idx = max(0, eq_index - window)
    end_idx = min(len(bond_distances), eq_index + window + 1)
    
    # Extract relevant data
    r_near_eq = bond_distances[start_idx:end_idx]
    e_near_eq = energies[start_idx:end_idx]
    
    # Fit to parabola: E = a*(r-r_e)^2 + E_min
    def parabola(r, a, r_e, E_min):
        return a * (r - r_e)**2 + E_min
    
    # Initial guesses
    p0 = [1.0, eq_distance, eq_energy]
    
    # Fit the data
    params, _ = curve_fit(parabola, r_near_eq, e_near_eq, p0=p0)
    a, r_e, E_min = params
    
    # Calculate force constant k
    # E = 0.5 * k * (r - r_e)^2 => k = 2a
    # Convert units: 1 Hartree/Å² = 627.5 kcal/(mol*Å²) = 627.5*4184/1e-20 = 2.625e26 J/m²
    k = 2 * a 
    k_J_per_m2 = k * 627.5 * 4184 / 1e-20
    
    # Calculate vibrational frequency
    # ω = sqrt(k/μ), μ = reduced mass = (m1*m2)/(m1+m2) = m_H/2 for H2
    m_H = 1.00794  # amu
    mu = m_H / 2   # amu
    mu_kg = mu * 1.66053886e-27  # kg
    
    # Angular frequency in rad/s
    omega = np.sqrt(k_J_per_m2 / mu_kg)
    
    # Convert to frequency in Hz and wavenumbers (cm⁻¹)
    nu = omega / (2 * np.pi)
    wavenumber = nu / 2.99792458e10
    
    print("\nHarmonic Oscillator Analysis:")
    print(f"Force constant (k): {k:.4f} Hartree/Å² = {k*627.5:.1f} kcal/(mol·Å²)")
    print(f"Vibrational frequency: {wavenumber:.1f} cm⁻¹")
    
    # Compare to literature value
    lit_value = 4401  # cm⁻¹ for H2
    error = abs(wavenumber - lit_value) / lit_value * 100
    print(f"Literature value: {lit_value} cm⁻¹")
    print(f"Percentage error: {error:.1f}%")
    
except Exception as e:
    print(f"\nCould not perform vibrational analysis: {str(e)}")

# 🔬 Advanced Quantum Chemistry

In this section, we'll explore more advanced calculations:
1. Molecular orbitals and electron density
2. Different basis sets and their effects
3. Integration with machine learning

The code will automatically adapt based on the available packages in your environment.

In [ ]:
# Molecular Orbital Visualization

import numpy as np
import matplotlib.pyplot as plt
from importlib.util import find_spec
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

print("🧪 Molecular Orbital Calculation and Visualization")
print("=" * 60)

# Define water molecule - more interesting orbitals than H2
water_geometry = """
O 0.0 0.0 0.0
H 0.0 0.757 0.586
H 0.0 -0.757 0.586
"""

# Function to calculate and visualize molecular orbitals
def calculate_molecular_orbitals(geometry, basis='sto-3g'):
    """Calculate molecular orbitals using the best available method"""
    
    if find_spec("psi4"):
        print("Using Psi4 for molecular orbital calculation...")
        import psi4
        psi4.set_output_file('molecular_orbitals.out')
        
        # Create molecule
        mol = psi4.geometry(geometry)
        
        # Set options
        psi4.set_options({"basis": basis, "cubeprop_tasks": ["orbitals"]})
        
        # Run calculation and get wavefunction
        energy, wfn = psi4.energy('scf', return_wfn=True)
        
        # Get orbital data
        Ca = wfn.Ca()
        Da = wfn.Da()
        
        # Get orbital energies (eigenvalues)
        epsilon_a = np.array(wfn.epsilon_a())
        
        # Get number of electrons
        nalpha = wfn.nalpha()
        
        print(f"Number of occupied alpha orbitals: {nalpha}")
        print(f"Number of basis functions: {Ca.shape[0]}")
        
        # Return orbital information
        return {
            "energies": epsilon_a,
            "nalpha": nalpha,
            "nbasis": Ca.shape[0],
            "method": "psi4",
            "energy": energy
        }
        
    elif find_spec("pyscf"):
        print("Using PySCF for molecular orbital calculation...")
        from pyscf import gto, scf
        
        # Create molecule
        mol = gto.M(atom=geometry, basis=basis, verbose=0)
        
        # Run calculation
        mf = scf.RHF(mol)
        mf.kernel()
        
        # Get orbital energies (eigenvalues) and coefficients
        mo_energy = mf.mo_energy
        mo_coeff = mf.mo_coeff
        
        # Get number of electrons
        nalpha = mol.nelec[0]  # Alpha electrons
        
        print(f"Number of occupied alpha orbitals: {nalpha}")
        print(f"Number of basis functions: {len(mo_energy)}")
        
        # Return orbital information
        return {
            "energies": mo_energy,
            "nalpha": nalpha,
            "nbasis": len(mo_energy),
            "method": "pyscf",
            "energy": mf.e_tot,
            "mf": mf,
            "mol": mol
        }
    
    else:
        print("Using mock data for molecular orbitals (no quantum chemistry packages available)")
        # Mock data for water STO-3G
        mock_energies = np.array([-20.55, -1.34, -0.71, -0.57, -0.50, 0.21, 0.30])
        nalpha = 5  # Water has 10 electrons total, 5 alpha
        
        # Return mock data
        return {
            "energies": mock_energies,
            "nalpha": nalpha,
            "nbasis": len(mock_energies),
            "method": "mock",
            "energy": -76.0266  # Mock total energy for water
        }

# Calculate molecular orbitals
print("Calculating molecular orbitals for water molecule...")
orbital_data = calculate_molecular_orbitals(water_geometry)

# Extract data from results
orbital_energies = orbital_data["energies"]
num_occupied = orbital_data["nalpha"]
method_used = orbital_data["method"]
total_energy = orbital_data["energy"]

print(f"Calculation completed using {method_used.upper()}")
print(f"Total energy: {total_energy:.6f} Hartree")
print(f"Number of basis functions: {orbital_data['nbasis']}")
print(f"Number of occupied orbitals: {num_occupied}")

# Plot orbital energy diagram with annotations
plt.figure(figsize=(10, 8))

# Plot vertical line to represent energy axis
plt.plot([0, 0], [min(orbital_energies)-0.5, max(orbital_energies)+0.5], 'k-', alpha=0.3)

# Plot orbital energy levels with better styling
homo_idx = num_occupied - 1
for i, energy in enumerate(orbital_energies):
    # Set color and style based on occupation
    if i <= homo_idx:
        color = 'blue'
        label = 'Occupied' if i == 0 else None
        marker = 'o'
        linestyle = '-'
    else:
        color = 'red'
        label = 'Virtual' if i == homo_idx + 1 else None
        marker = 's'
        linestyle = '--'
    
    # Highlight HOMO and LUMO
    if i == homo_idx:
        color = 'darkblue'
        label = 'HOMO'
        marker = '*'
        markersize = 15
    elif i == homo_idx + 1:
        color = 'darkred'
        label = 'LUMO'
        marker = '*'
        markersize = 15
    else:
        markersize = 10
    
    # Plot horizontal line for each orbital
    plt.plot([-0.2, 0.2], [energy, energy], linestyle=linestyle, linewidth=2, color=color, label=label)
    plt.plot(0, energy, marker=marker, markersize=markersize, color=color)
    
    # Add orbital number and energy
    plt.text(-0.5, energy, f"MO {i+1}", fontsize=10, ha='center', va='center')
    plt.text(0.5, energy, f"{energy:.2f} Ha", fontsize=10, ha='left', va='center')

# Remove duplicate labels
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), loc='best')

# Calculate and display HOMO-LUMO gap
homo_energy = orbital_energies[homo_idx]
lumo_energy = orbital_energies[homo_idx + 1]
gap = lumo_energy - homo_energy

# Add arrow and label for HOMO-LUMO gap
plt.annotate(f"Gap: {gap:.2f} Ha\n({gap*27.2114:.2f} eV)", 
             xy=(0, homo_energy + gap/2),
             xytext=(0.7, homo_energy + gap/2),
             arrowprops=dict(arrowstyle='<->', color='purple'),
             fontsize=12, color='purple')

# Set plot parameters
plt.title(f"Molecular Orbital Energies for Water ({method_used.upper()})", fontsize=14)
plt.ylabel('Orbital Energy (Hartree)', fontsize=12)
plt.grid(True, axis='y', alpha=0.3)
plt.xlim(-0.6, 1.2)
plt.ylim(min(orbital_energies) - 0.5, max(orbital_energies) + 0.5)

# Remove x-axis ticks and label
plt.xticks([])
plt.tight_layout()
plt.show()

# Print HOMO-LUMO gap with conversions to different units
print(f"\nHOMO-LUMO gap: {gap:.4f} Hartree")
print(f"                {gap*27.2114:.2f} eV")
print(f"                {gap*627.5:.2f} kcal/mol")
print(f"                {gap*2625.5:.2f} kJ/mol")

# Print orbital energies
print("\nOrbital energies (Hartree):")
for i, energy in enumerate(orbital_energies):
    status = "Occupied" if i < num_occupied else "Virtual"
    if i == homo_idx:
        status += " (HOMO)"
    elif i == homo_idx + 1:
        status += " (LUMO)"
    print(f"MO {i+1}: {energy:.6f} ({status})")

# If using PySCF, try to visualize the orbitals more directly
if method_used == "pyscf" and "mf" in orbital_data and "mol" in orbital_data:
    try:
        print("\nAttempting to visualize molecular orbitals using PySCF...")
        
        from pyscf import tools
        import matplotlib.pyplot as plt
        from mpl_toolkits.mplot3d import Axes3D
        
        mf = orbital_data["mf"]
        mol = orbital_data["mol"]
        
        # Create a grid for visualization
        nx, ny, nz = 40, 40, 40
        grid = np.mgrid[-3:3:nx*1j, -3:3:ny*1j, -3:3:nz*1j]
        coords = np.vstack([grid[0].ravel(), grid[1].ravel(), grid[2].ravel()]).T
        
        # Choose orbitals to visualize (HOMO and LUMO)
        orbitals_to_vis = [homo_idx, homo_idx + 1]  # 0-indexed
        labels = ["HOMO", "LUMO"]
        
        # Create subplots
        fig = plt.figure(figsize=(12, 5))
        
        for idx, (orbital_idx, label) in enumerate(zip(orbitals_to_vis, labels)):
            # Get the orbital values on the grid
            ao_value = tools.cubegen.orbital(mol, coords, mf.mo_coeff[:, orbital_idx])
            ao_value = ao_value.reshape(nx, ny, nz)
            
            # Plot the orbital in z=0 plane
            ax = fig.add_subplot(1, 2, idx+1)
            
            # Create contour plot
            z_mid = nz // 2
            contour = ax.contourf(grid[0][:,:,z_mid], grid[1][:,:,z_mid], 
                               ao_value[:,:,z_mid], 
                               levels=50, cmap='RdBu')
            
            # Add atom positions
            for atom in range(mol.natm):
                x, y, z = mol.atom_coord(atom)
                if abs(z) < 1.0:  # Only show atoms near the z=0 plane
                    ax.plot(x, y, 'o', markersize=10, color='k')
                    symbol = mol.atom_symbol(atom)
                    ax.text(x+0.1, y+0.1, symbol, fontsize=12)
            
            # Add title and labels
            ax.set_title(f"{label} (MO {orbital_idx+1})", fontsize=12)
            ax.set_xlabel('Å')
            ax.set_ylabel('Å')
            
            # Add colorbar
            plt.colorbar(contour, ax=ax)
        
        plt.tight_layout()
        plt.show()
        print("Molecular orbital visualization complete!")
    except Exception as e:
        print(f"Could not visualize orbitals with PySCF: {str(e)}")

In [ ]:
# Exploring Different Basis Sets

import numpy as np
import matplotlib.pyplot as plt
from importlib.util import find_spec
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

print("🧪 Effect of Basis Sets on Energy Calculation")
print("=" * 60)

# Define our test molecule - water
water_geometry = """
O 0.0 0.0 0.0
H 0.0 0.757 0.586
H 0.0 -0.757 0.586
"""

# Define a dictionary of basis sets with their information
basis_sets_info = {
    'sto-3g': {'description': 'Minimal basis set', 'size': 'Smallest', 'functions_per_atom': 'H:1, C:5'},
    'sto-6g': {'description': 'Minimal basis set with more Gaussians', 'size': 'Small', 'functions_per_atom': 'H:1, C:5'},
    '3-21g': {'description': 'Split-valence double-zeta', 'size': 'Medium-small', 'functions_per_atom': 'H:2, C:9'}, 
    '6-31g': {'description': 'Split-valence double-zeta', 'size': 'Medium', 'functions_per_atom': 'H:2, C:9'},
    '6-31g*': {'description': 'Double-zeta with polarization on heavy atoms', 'size': 'Medium', 'functions_per_atom': 'H:2, C:15'},
    '6-311g': {'description': 'Triple-zeta basis', 'size': 'Medium-large', 'functions_per_atom': 'H:3, C:13'},
    'cc-pvdz': {'description': 'Correlation-consistent double-zeta', 'size': 'Medium-large', 'functions_per_atom': 'H:5, C:14'},
    'cc-pvtz': {'description': 'Correlation-consistent triple-zeta', 'size': 'Large', 'functions_per_atom': 'H:14, C:30'},
    'aug-cc-pvdz': {'description': 'Augmented cc-pVDZ (with diffuse functions)', 'size': 'Large', 'functions_per_atom': 'H:9, C:23'},
}

# Select basis sets to test (ordered by increasing size/quality)
basis_sets = ['sto-3g', '3-21g', '6-31g', 'cc-pvdz', 'cc-pvtz']
energies = []
successful_basis = []
method_used = None

# Display basis set information
print("Basis sets to be tested (in order of increasing size/quality):")
for i, basis in enumerate(basis_sets):
    if basis in basis_sets_info:
        info = basis_sets_info[basis]
        print(f"{i+1}. {basis}: {info['description']} ({info['size']})")
    else:
        print(f"{i+1}. {basis}")

print("\nStarting calculations...")

# Try calculations with different basis sets
for basis in basis_sets:
    print(f"\nTrying basis set: {basis}...")
    try:
        energy, method = calculate_energy(water_geometry, basis)
        energies.append(energy)
        successful_basis.append(basis)
        print(f"  ✅ Success! Energy = {energy:.6f} Hartree")
        
        if method_used is None:
            method_used = method
    except Exception as e:
        print(f"  ❌ Failed with basis {basis}: {str(e)}")

if len(energies) > 0:
    print(f"\nComparison completed using {method_used.upper()}")
    
    # Convert energies to array
    energies = np.array(energies)
    
    # Convert to relative energies (absolute values) in kcal/mol
    e_min = min(energies)
    relative_energies = (energies - e_min) * 627.5
    
    # Create a DataFrame for nicer display
    basis_data = pd.DataFrame({
        'Basis Set': successful_basis,
        'Energy (Hartree)': energies,
        'Relative (kcal/mol)': relative_energies,
        'Basis Size': [basis_sets_info.get(b, {}).get('size', 'Unknown') for b in successful_basis]
    })
    
    # Estimate the Hartree-Fock limit using exponential extrapolation
    try:
        from scipy.optimize import curve_fit
        
        # Define extrapolation function: E(x) = E_inf + A*exp(-B*x)
        def extrapolation_function(x, e_inf, a, b):
            return e_inf + a * np.exp(-b * x)
        
        # Use basis set index as x (simplified approach)
        x_data = np.arange(1, len(energies) + 1)
        
        # Fit the curve
        params, _ = curve_fit(extrapolation_function, x_data, energies)
        
        # Extract parameters
        e_inf, a, b = params
        
        # Calculate extrapolated values
        x_extended = np.arange(1, len(energies) + 3)
        y_extended = extrapolation_function(x_extended, e_inf, a, b)
        
        # Add extrapolated values to DataFrame
        if len(successful_basis) >= 3:
            basis_data = basis_data.append({
                'Basis Set': 'Extrapolated',
                'Energy (Hartree)': e_inf,
                'Relative (kcal/mol)': (e_inf - e_min) * 627.5,
                'Basis Size': 'Infinite'
            }, ignore_index=True)
        
        has_extrapolation = True
    except:
        has_extrapolation = False
        print("Could not perform basis set extrapolation")
    
    # Display DataFrame
    from IPython.display import display
    print("\nBasis Set Comparison:")
    display(basis_data)
    
    # Create visualization
    plt.figure(figsize=(12, 6))
    
    # Plot basis set convergence
    plt.subplot(1, 2, 1)
    plt.plot(range(len(successful_basis)), energies, 'o-', markersize=8, color='blue')
    
    if has_extrapolation:
        plt.plot(len(successful_basis), e_inf, '*', markersize=12, color='red')
        plt.axhline(y=e_inf, linestyle='--', color='red', alpha=0.5, 
                   label=f'Estimated limit: {e_inf:.6f} Ha')
    
    plt.xticks(range(len(successful_basis)), successful_basis, rotation=45)
    plt.ylabel('Energy (Hartree)', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.title('Basis Set Convergence', fontsize=14)
    if has_extrapolation:
        plt.legend()
    
    # Plot relative energies
    plt.subplot(1, 2, 2)
    bars = plt.bar(successful_basis, relative_energies, color='skyblue', alpha=0.7)
    
    # Add energy values on top of bars
    for i, (bar, energy) in enumerate(zip(bars, relative_energies)):
        plt.text(i, energy + 0.1, f"{energy:.2f}", ha='center')
    
    plt.axhline(y=0, color='red', linestyle='--', alpha=0.7)
    plt.ylabel('Relative Energy (kcal/mol)', fontsize=12)
    plt.title('Energy Relative to Minimum', fontsize=14)
    plt.grid(True, axis='y', alpha=0.3)
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Create table for comparison
    print("\nBasis Set Comparison Table:")
    print(f"{'Basis Set':<12} {'Energy (Hartree)':<20} {'Relative (kcal/mol)':<20} {'Improvement (kcal/mol)':<25}")
    print("-" * 75)
    
    prev_energy = None
    for i, basis in enumerate(successful_basis):
        energy = energies[i]
        rel_energy = relative_energies[i]
        
        if prev_energy is not None:
            improvement = (prev_energy - energy) * 627.5
            improvement_str = f"{improvement:.2f}"
        else:
            improvement_str = "N/A"
        
        print(f"{basis:<12} {energy:<20.6f} {rel_energy:<20.2f} {improvement_str:<25}")
        prev_energy = energy
    
    if has_extrapolation and len(successful_basis) >= 3:
        improvement = (energies[-1] - e_inf) * 627.5
        print(f"{'Extrap.':<12} {e_inf:<20.6f} {(e_inf - e_min) * 627.5:<20.2f} {improvement:.2f}")
    
    # Print interpretation
    print("\nInterpretation:")
    print("1. Larger basis sets generally yield lower (more negative) energies")
    print("2. The energy converges as the basis set size increases")
    print("3. Each improvement in basis set quality yields diminishing returns")
    if has_extrapolation and len(successful_basis) >= 3:
        print(f"4. The estimated complete basis set limit is: {e_inf:.6f} Hartree")
    
    # Calculate computational cost scaling
    if len(successful_basis) >= 2:
        print("\nComputational Scaling Estimates:")
        print("Formal scaling of Hartree-Fock is O(N^4) where N is the number of basis functions")
        for i in range(min(3, len(successful_basis))):
            basis = successful_basis[i]
            if basis in basis_sets_info:
                size = basis_sets_info[basis]['size']
                print(f"- {basis}: {size} basis set")
                
else:
    print("\n❌ No successful basis set calculations")

# 🤖 Machine Learning with Quantum Chemistry Data

In this section, we'll demonstrate how machine learning can be combined with quantum chemistry. We'll:

1. Generate quantum chemistry data for a set of small molecules
2. Extract features from the calculations
3. Train a simple ML model to predict properties
4. Evaluate and visualize the results

This section showcases the power of combining quantum chemistry with data science.

In [ ]:
# Machine Learning with Quantum Chemistry Data

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib.util import find_spec
import warnings
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.pipeline import Pipeline

# Suppress warnings
warnings.filterwarnings('ignore')

print("🤖 Machine Learning with Quantum Chemistry Data")
print("=" * 60)

# Create a dataset of simple molecules with experimental bond energies (kcal/mol)
molecules = [
    {"formula": "H2", "smiles": "[H][H]", "geometry": "H 0 0 0\nH 0 0 0.74", "bond_energy": 104.2},
    {"formula": "N2", "smiles": "N#N", "geometry": "N 0 0 0\nN 0 0 1.10", "bond_energy": 225.1},
    {"formula": "O2", "smiles": "O=O", "geometry": "O 0 0 0\nO 0 0 1.21", "bond_energy": 119.1},
    {"formula": "F2", "smiles": "F-F", "geometry": "F 0 0 0\nF 0 0 1.42", "bond_energy": 36.6},
    {"formula": "Cl2", "smiles": "Cl-Cl", "geometry": "Cl 0 0 0\nCl 0 0 1.99", "bond_energy": 58.0},
    {"formula": "Br2", "smiles": "Br-Br", "geometry": "Br 0 0 0\nBr 0 0 2.28", "bond_energy": 46.1},
    {"formula": "I2", "smiles": "I-I", "geometry": "I 0 0 0\nI 0 0 2.67", "bond_energy": 36.1},
    {"formula": "CO", "smiles": "C#O", "geometry": "C 0 0 0\nO 0 0 1.13", "bond_energy": 257.3},
    {"formula": "HF", "smiles": "F", "geometry": "H 0 0 0\nF 0 0 0.92", "bond_energy": 135.8},
    {"formula": "HCl", "smiles": "Cl", "geometry": "H 0 0 0\nCl 0 0 1.27", "bond_energy": 103.2},
    {"formula": "HBr", "smiles": "Br", "geometry": "H 0 0 0\nBr 0 0 1.41", "bond_energy": 87.5},
    {"formula": "HI", "smiles": "I", "geometry": "H 0 0 0\nI 0 0 1.61", "bond_energy": 71.3},
    {"formula": "CO2", "smiles": "O=C=O", "geometry": "C 0 0 0\nO 1.16 0 0\nO -1.16 0 0", "bond_energy": 192.0}
]

# Function to calculate quantum features for a molecule
def calculate_quantum_features(molecule, basis="sto-3g"):
    """Calculate quantum chemical features for ML input"""
    
    try:
        # Calculate energy with error handling
        try:
            energy, method = calculate_energy(molecule["geometry"], basis)
        except Exception as e:
            print(f"  Energy calculation failed: {str(e)}")
            return None, False
        
        # Extract simple features based on available methods
        features = {
            "energy": energy,
            "formula": molecule["formula"],
            "bond_energy": molecule["bond_energy"],
            "num_atoms": len(molecule["geometry"].strip().split('\n'))
        }
        
        # Try to get more features if possible
        if method == "psi4" and find_spec("psi4"):
            print(f"  Using Psi4 for advanced features...")
            import psi4
            # Run calculation and get wavefunction
            try:
                mol = psi4.geometry(molecule["geometry"])
                psi4.set_options({"basis": basis})
                energy, wfn = psi4.energy('scf', return_wfn=True)
                
                # Add dipole moment
                features["dipole"] = np.linalg.norm(np.array(wfn.dipole()))
                
                # Add HOMO-LUMO gap
                eps_a = np.array(wfn.epsilon_a())
                homo_idx = wfn.nalpha() - 1
                features["homo_lumo_gap"] = eps_a[homo_idx + 1] - eps_a[homo_idx]
                
                # Add total electronic energy components
                features["nuclear_repulsion"] = mol.nuclear_repulsion_energy()
                features["one_electron_energy"] = wfn.one_electron_energy()
                features["two_electron_energy"] = wfn.two_electron_energy()
            except Exception as e:
                print(f"  Psi4 advanced features failed: {str(e)}")
            
        elif method == "pyscf" and find_spec("pyscf"):
            print(f"  Using PySCF for advanced features...")
            try:
                from pyscf import gto, scf
                # Create molecule
                mol = gto.M(atom=molecule["geometry"], basis=basis, verbose=0)
                
                # Run calculation
                mf = scf.RHF(mol)
                mf.kernel()
                
                # Add dipole moment if available
                try:
                    dip = mf.dip_moment()
                    features["dipole"] = np.linalg.norm(dip)
                except:
                    features["dipole"] = 0.0
                
                # Add HOMO-LUMO gap
                mo_energy = mf.mo_energy
                homo_idx = mol.nelec[0] - 1
                features["homo_lumo_gap"] = mo_energy[homo_idx + 1] - mo_energy[homo_idx]
                
                # Add energy components
                features["nuclear_repulsion"] = mol.energy_nuc()
                features["one_electron_energy"] = np.trace(mf.get_hcore() @ mf.make_rdm1())
                features["two_electron_energy"] = 0.5 * np.einsum('ij,ji->', mf.get_veff(), mf.make_rdm1())
            except Exception as e:
                print(f"  PySCF advanced features failed: {str(e)}")
        else:
            # Add mock features with reasonable values
            if "H2" in molecule["formula"]:
                features["dipole"] = 0.0
                features["homo_lumo_gap"] = 0.5
                features["nuclear_repulsion"] = 0.7
                features["one_electron_energy"] = -2.5
                features["two_electron_energy"] = 0.7
            elif "N2" in molecule["formula"]:
                features["dipole"] = 0.0
                features["homo_lumo_gap"] = 0.4
                features["nuclear_repulsion"] = 23.5
                features["one_electron_energy"] = -110.5
                features["two_electron_energy"] = 45.2
            else:
                # Generate somewhat random but reasonable values
                import random
                features["dipole"] = random.uniform(0, 2.0) if "C" in molecule["formula"] else 0.0
                features["homo_lumo_gap"] = random.uniform(0.3, 0.7)
                features["nuclear_repulsion"] = random.uniform(10, 30)
                features["one_electron_energy"] = random.uniform(-120, -80)
                features["two_electron_energy"] = random.uniform(30, 60)
        
        # Add more atomic property-based features
        try:
            # Try to add atomic properties-based features using Python's periodic table
            # Package like mendeleev, periodictable, or simple lookup tables
            if find_spec("mendeleev"):
                import mendeleev
                
                # Extract element symbols
                elements = [line.strip().split()[0] for line in molecule["geometry"].strip().split('\n')]
                
                # Calculate average properties
                electronegativities = []
                ionization_energies = []
                
                for elem in elements:
                    try:
                        e = mendeleev.element(elem)
                        if e.electronegativity is not None:
                            electronegativities.append(e.electronegativity)
                        if e.ionenergies is not None and 1 in e.ionenergies:
                            ionization_energies.append(e.ionenergies[1])
                    except:
                        pass
                
                if electronegativities:
                    features["avg_electronegativity"] = np.mean(electronegativities)
                    features["max_electronegativity"] = np.max(electronegativities)
                    features["min_electronegativity"] = np.min(electronegativities)
                    
                if ionization_energies:
                    features["avg_ionization_energy"] = np.mean(ionization_energies)
                    features["max_ionization_energy"] = np.max(ionization_energies)
            else:
                # Use a simple lookup table for common elements
                electronegativity = {"H": 2.2, "C": 2.55, "N": 3.04, "O": 3.44, 
                                     "F": 3.98, "Cl": 3.16, "Br": 2.96, "I": 2.66}
                
                # Extract element symbols
                elements = [line.strip().split()[0] for line in molecule["geometry"].strip().split('\n')]
                
                # Get values
                en_values = [electronegativity.get(e, 2.0) for e in elements]
                
                # Calculate statistics
                if en_values:
                    features["avg_electronegativity"] = np.mean(en_values)
                    features["max_electronegativity"] = np.max(en_values)
                    features["min_electronegativity"] = np.min(en_values)
                    if len(en_values) > 1:
                        features["en_diff"] = np.max(en_values) - np.min(en_values)
        except Exception as e:
            print(f"  Error adding atomic properties: {str(e)}")
        
        # Extract bond length from geometry for diatomics
        if features["num_atoms"] == 2:
            try:
                lines = molecule["geometry"].strip().split('\n')
                atom1 = np.array([float(x) for x in lines[0].strip().split()[1:4]])
                atom2 = np.array([float(x) for x in lines[1].strip().split()[1:4]])
                bond_length = np.linalg.norm(atom1 - atom2)
                features["bond_length"] = bond_length
            except Exception as e:
                print(f"  Error calculating bond length: {str(e)}")
        
        return features, True
    
    except Exception as e:
        print(f"Failed to calculate features for {molecule['formula']}: {str(e)}")
        return None, False

# Calculate features for all molecules
print("Calculating quantum features for molecules...")
all_features = []
successful_count = 0

for molecule in molecules:
    print(f"Processing {molecule['formula']}...")
    features, success = calculate_quantum_features(molecule)
    if success:
        all_features.append(features)
        successful_count += 1
        print(f"  ✅ Success: Generated {len(features)} features")
    else:
        print(f"  ❌ Failed to generate features")

print(f"\nSuccessfully calculated features for {successful_count}/{len(molecules)} molecules")

# Create DataFrame from features
if successful_count > 0:
    df = pd.DataFrame(all_features)
    print("\nFeatures calculated (first few rows):")
    display(df.head())
    
    print(f"\nStatistical summary of numerical features:")
    display(df.describe())
    
    # Prepare data for ML
    if len(df) >= 5:  # Need enough data points for train/test split
        # Define features and target
        print("\nPreparing data for machine learning...")
        feature_cols = [col for col in df.columns if col not in ['formula', 'bond_energy']]
        print(f"Using {len(feature_cols)} features: {', '.join(feature_cols)}")
        
        X = df[feature_cols].fillna(-1)  # Replace any NaN with -1
        y = df['bond_energy']
        
        # Split data for training and testing
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        print(f"Split into {len(X_train)} training samples and {len(X_test)} testing samples")
        
        # Define ML models to try
        models = [
            ('Linear Regression', LinearRegression()),
            ('Ridge Regression', Ridge(alpha=1.0)),
            ('Random Forest', RandomForestRegressor(n_estimators=100, random_state=42)),
        ]
        
        # Try to add more advanced models if available
        if find_spec("xgboost"):
            from xgboost import XGBRegressor
            models.append(('XGBoost', XGBRegressor(n_estimators=100, random_state=42)))
        
        if find_spec("lightgbm"):
            from lightgbm import LGBMRegressor
            models.append(('LightGBM', LGBMRegressor(n_estimators=100, random_state=42)))
            
        # Test each model with scaling
        results = []
        best_model = None
        best_score = -np.inf
        best_name = None
        
        print("\nEvaluating models:")
        for name, model in models:
            # Create a pipeline with scaling
            pipeline = Pipeline([
                ('scaler', StandardScaler()),
                ('model', model)
            ])
            
            # Train model
            pipeline.fit(X_train, y_train)
            
            # Make predictions
            y_pred = pipeline.predict(X_test)
            
            # Evaluate
            mae = mean_absolute_error(y_test, y_pred)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            r2 = r2_score(y_test, y_pred)
            
            # Add to results
            results.append({
                'Model': name,
                'MAE (kcal/mol)': mae,
                'RMSE (kcal/mol)': rmse,
                'R²': r2
            })
            
            # Check if this is the best model
            if r2 > best_score:
                best_score = r2
                best_model = pipeline
                best_name = name
            
            print(f"  {name}: MAE = {mae:.2f} kcal/mol, R² = {r2:.2f}")
        
        # Display results as a table
        results_df = pd.DataFrame(results)
        print("\nModel Comparison:")
        display(results_df)
        
        # Use best model for further analysis
        print(f"\nUsing best model ({best_name}) for further analysis")
        
        # Create feature importance plot
        plt.figure(figsize=(12, 5))
        
        # Extract feature importance differently based on model type
        if hasattr(best_model['model'], 'feature_importances_'):
            # Tree-based models have feature_importances_
            feature_importance = best_model['model'].feature_importances_
            indices = np.argsort(feature_importance)[::-1]
            
            # Plot feature importances
            plt.subplot(1, 2, 1)
            plt.bar(range(X.shape[1]), feature_importance[indices], color='skyblue', alpha=0.7)
            plt.xticks(range(X.shape[1]), [feature_cols[i] for i in indices], rotation=90)
            plt.xlabel('Features')
            plt.ylabel('Importance')
            plt.title(f'Feature Importance ({best_name})')
            plt.tight_layout()
        elif hasattr(best_model['model'], 'coef_'):
            # Linear models have coef_
            coefficients = best_model['model'].coef_
            
            # Get absolute values and normalize
            abs_coeffs = np.abs(coefficients)
            if sum(abs_coeffs) > 0:  # Avoid division by zero
                norm_coeffs = abs_coeffs / sum(abs_coeffs)
            else:
                norm_coeffs = abs_coeffs
                
            indices = np.argsort(norm_coeffs)[::-1]
            
            # Plot normalized coefficients
            plt.subplot(1, 2, 1)
            plt.bar(range(X.shape[1]), norm_coeffs[indices], color='skyblue', alpha=0.7)
            plt.xticks(range(X.shape[1]), [feature_cols[i] for i in indices], rotation=90)
            plt.xlabel('Features')
            plt.ylabel('Normalized |Coefficient|')
            plt.title(f'Feature Importance ({best_name})')
            plt.tight_layout()
        
        # Plot actual vs predicted
        plt.subplot(1, 2, 2)
        y_pred = best_model.predict(X_test)
        plt.scatter(y_test, y_pred, alpha=0.7, c='blue')
        
        # Add perfect prediction line
        min_val = min(min(y_test), min(y_pred))
        max_val = max(max(y_test), max(y_pred))
        plt.plot([min_val, max_val], [min_val, max_val], 'r--')
        
        # Add error bands for ±5 kcal/mol
        plt.fill_between([min_val, max_val], [min_val-5, max_val-5], [min_val+5, max_val+5],
                         color='red', alpha=0.1, label='±5 kcal/mol error')
        
        plt.xlabel('Actual Bond Energy (kcal/mol)')
        plt.ylabel('Predicted Bond Energy (kcal/mol)')
        plt.title(f'Actual vs Predicted Bond Energy\nR² = {best_score:.2f}')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.tight_layout()
        plt.show()
        
        # Try to visualize with a parity plot if possible
        try:
            # Create DataFrame with predictions
            results_df = pd.DataFrame({
                'Formula': df['formula'].iloc[y_test.index],
                'Actual': y_test,
                'Predicted': y_pred,
                'Error': y_test - y_pred
            })
            
            # Sort by error
            results_df = results_df.sort_values('Error')
            
            # Display results for each molecule
            print("\nPrediction results for test molecules:")
            display(results_df)
            
            # Create error histogram
            plt.figure(figsize=(10, 5))
            plt.hist(results_df['Error'], bins=10, alpha=0.7, color='skyblue')
            plt.axvline(0, color='red', linestyle='--', alpha=0.7)
            plt.xlabel('Prediction Error (kcal/mol)')
            plt.ylabel('Number of Molecules')
            plt.title('Distribution of Prediction Errors')
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()
        except Exception as e:
            print(f"Could not create additional visualizations: {str(e)}")
        
        # Print conclusions
        print("\nConclusions:")
        print(f"1. The best model ({best_name}) achieves {best_score:.2f} R² score")
        
        # Find most important feature
        if hasattr(best_model['model'], 'feature_importances_'):
            top_feature_idx = np.argmax(best_model['model'].feature_importances_)
            print(f"2. The most important feature is '{feature_cols[top_feature_idx]}'")
        elif hasattr(best_model['model'], 'coef_'):
            top_feature_idx = np.argmax(np.abs(best_model['model'].coef_))
            print(f"2. The most important feature is '{feature_cols[top_feature_idx]}'")
        
        # Error analysis
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        print(f"3. Mean Absolute Error: {mae:.2f} kcal/mol, RMSE: {rmse:.2f} kcal/mol")
        
        print("\nNext steps:")
        print("1. Collect more molecular data for better training")
        print("2. Engineer additional quantum-mechanical features")
        print("3. Try more advanced ML models like deep neural networks")
        print("4. Perform hyperparameter optimization for model tuning")
        
    else:
        print("\n❌ Not enough molecules for ML model training. Need at least 5 molecules.")
else:
    print("\n❌ No features calculated successfully")

# 🎓 Conclusion and Next Steps

In this notebook, we've explored the integration of quantum chemistry with Python. We demonstrated:

1. **Basic quantum calculations** - Single point energies and potential energy surfaces
2. **Advanced quantum analysis** - Molecular orbitals and basis set effects 
3. **Machine learning integration** - Using quantum features to predict molecular properties

The notebook was designed to work with or without specialized quantum chemistry packages, using fallback mechanisms when needed.

## 🚀 Next Steps

To continue learning quantum chemistry with Python:

1. **Explore more molecules** - Try more complex systems like benzene or amino acids
2. **Try advanced methods** - Explore MP2, CCSD, or DFT for more accurate calculations
3. **Improve ML models** - Add more features, try different algorithms like neural networks
4. **Quantum dynamics** - Study how molecules change over time with molecular dynamics

## 📚 Further Resources

- [Psi4 Documentation](https://psicode.org/psi4manual/master/index.html)
- [PySCF Documentation](https://pyscf.org/documentation.html)
- [RDKit Documentation](https://www.rdkit.org/docs/index.html)
- [DeepChem Documentation](https://deepchem.readthedocs.io/)